# Algoritmo de busca vetorial
Vamos transformar os resumos em embeddings para poder fazer uma busca vetorial em cima

In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
df = pd.read_parquet('dados/articles2.parquet')
df

,title,title_alt,authors,advisor1,keywords,date,publisher,campus,citation,resumo,abstract,uri,pdf_uri
unique_id,,,,,,,,,,,,,
28320,Validação de modelos utilizando problemas inve...,None,"[Silva, Vinícius Vitorelli Generoso da]","Koroishi, Edson Hideki","[Problemas inversos (Equações diferenciais), V...",2015-02-11,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"SILVA, Vinícius Vitorelli Generoso da. Validaç...",Existe uma dificuldade desde os primórdios da ...,From the beginning there is difficulty in engi...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36735,Destinação do lodo de estação de tratamento de...,Disposal of water treatment plant sludge in wa...,"[Tamankievies, Daniela Fatima]","Bortoli, Marcelo","[Água - Estações de tratamento, Lodo, Testes d...",2025-03-27,Universidade Tecnológica Federal do Paraná,Francisco Beltrao,"TAMANKIEVIES, Daniela Fatima. Destinação do lo...",O lodo gerado em Estações de Tratamento de Águ...,The sludge generated in Water Treatment Plants...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36437,Gramep: método livre de alinhamento baseado no...,Gramep: an alignment-free method based on the ...,"[Zanon, Matheus Henrique Pimenta]","Lopes, Fabricio Martins","[Genômica, Aprendizado do computador, Classifi...",2025-03-27,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"ZANON, Matheus Henrique Pimenta. Gramep: métod...",Com o avanço na tecnologia de sequenciamento d...,"With advances in DNA sequencing technology, th...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36652,Frações na perspectiva de medição e a abordage...,Fractions from the measurement perspective and...,"[Shiinoki, Vanessa Garcia]","Rocha, Zenaide de Fatima Dante Correia","[Matemática - Estudo e ensino, Frações, Mediçã...",2025-03-27,Universidade Tecnológica Federal do Paraná,Londrina,"SHIINOKI, Vanessa Garcia. Frações na perspecti...",O objetivo desta pesquisa foi analisar os proc...,The objective of this research was to analyze ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
37105,Consciência fonológica na alfabetização: propo...,Phonological awareness in the literacy: a prop...,"[Rosa, Daniela Dala]","Denardi, Didie Ana Ceni","[Língua inglesa, Fonologia, Literacia, Ensino ...",2025-03-28,Universidade Tecnológica Federal do Paraná,Pato Branco,"ROSA, Daniela Dala. Consciência fonológica na ...","Esta pesquisa explicatória (Gil, 2021) versa s...","This exploratory research (Gil, 2021) deals wi...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,Vibração livre e flambagem de placas de Mindli...,Free-vibration and buckling analysis of Mindli...,"[Araujo, Leilson Joaquim]","Abdalla Filho, João Elias","[Vibração, Método dos elementos finitos, Flamb...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"ARAUJO, Leilson Joaquim. Vibração livre e flam...",O método dos elementos finitos (MEF) é um méto...,The finite element method (FEM) is an approxim...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
25542,Equilíbrio de fases experimental de hidratos d...,Experimental phase equilibria of carbon dioxid...,"[Vasconcelos, Luiz Fernando Santos de]","Marcelino Neto, Moises Alves","[Hidratos, Dióxido de carbono, Metano, Cloreto...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"VASCONCELOS, Luiz Fernando Santos de. Equilíbr...","Em certas condições termodinâmicas, as molécul...","Under certain thermodynamic conditions, water ...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
26006,Influência das etapas de branqueamento e tempe...,Influence of bleaching steps 

In [10]:
df.head().to_dict('records')[0]

{'title': 'Validação de modelos utilizando problemas inversos',
 'title_alt': None,
 'authors': array(['Silva, Vinícius Vitorelli Generoso da'], dtype=object),
 'advisor1': 'Koroishi, Edson Hideki',
 'keywords': array(['Problemas inversos (Equações diferenciais)', 'Vigas', 'Vibração',
        'Inverse problems (Differential equations)', 'Girders',
        'Vibration'], dtype=object),
 'date': Timestamp('2015-02-11 00:00:00'),
 'publisher': 'Universidade Tecnológica Federal do Paraná',
 'campus': 'Cornelio Procopio',
 'citation': 'SILVA, Vinícius Vitorelli Generoso da. Validação de modelos utilizando problemas inversos. 2015. Trabalho de Conclusão de Curso (Bacharelado em Engenharia Mecânica) - Universidade Tecnológica Federal do Paraná, Cornélio Procópio, 2015.',
 'resumo': 'Existe uma dificuldade desde os primórdios da engenharia em obter-se algumas grandezas físicas e propriedades geométricas de diversas aplicações. A abordagem de problemas inversos possibilita o estudo das causas co

In [3]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
chunk_size = 4
df_chunks = np.array_split(df, math.ceil(len(df)/chunk_size))
for i, chunk in tqdm(enumerate(df_chunks), total=len(df_chunks)):
    embeddings = model.encode(list(chunk['resumo'].values), batch_size=chunk_size, max_length=8192, return_dense=True, return_sparse=False, return_colbert_vecs=False)['dense_vecs']
    embeddings_df = pd.concat([chunk.reset_index()['unique_id'], pd.DataFrame(embeddings)], axis=1)
    output_path='dados/embeddings.csv'
    embeddings_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

/home/guilherme/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
  0%|                                       | 3/4445 [01:44<43:04:08, 34.91s/it]


In [6]:
df_emb = pd.read_csv('dados/embeddings.csv', index_col='unique_id')
df_emb = df_emb.drop_duplicates()
df_emb

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
unique_id,,,,,,,,,,,,,,,,,,,,,
28320,-0.019500,-0.001077,-0.02010,0.024440,-0.032100,-0.013504,0.031200,0.022630,-0.003990,0.04320,...,0.027340,0.006676,0.02670,0.031740,0.013240,0.018080,0.01438,0.013350,-0.013290,-0.032070
36735,0.041840,-0.014670,-0.05762,0.036100,0.008460,-0.090200,-0.044600,0.042080,-0.020950,0.03104,...,0.016340,-0.011550,-0.00843,-0.001452,0.006560,0.000865,-0.01087,-0.030260,0.001152,0.012970
36437,-0.059630,-0.039030,-0.05127,0.031080,-0.030330,-0.036200,-0.010414,0.003754,0.016340,0.01756,...,0.001934,-0.002138,0.06097,-0.022930,0.002940,0.017610,-0.05460,0.016170,0.019420,0.034450
36652,0.016330,0.004032,-0.04710,0.002224,-0.021120,-0.009020,-0.004356,0.041930,0.001836,0.01639,...,0.005836,0.022660,0.03156,-0.014400,0.049130,0.001088,-0.01406,0.002285,0.016630,0.043370
37105,-0.046880,0.005180,0.00521,-0.053620,-0.011040,-0.032470,-0.011010,0.056430,0.000260,-0.01164,...,0.020940,-0.002071,0.03867,-0.018280,0.030470,-0.035370,0.07830,-0.009340,-0.002449,0.051200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,0.001677,-0.057770,-0.04578,-0.027280,0.026750,-0.079100,0.050750,0.028440,-0.011400,0.02255,...,0.049680,-0.008110,0.03564,0.016970,0.044770,-0.019940,0.04672,-0.050720,-0.006187,0.061950
25542,0.004005,-0.027190,-0.02230,-0.008010,0.021360,-0.039520,-0.016390,0.023240,-0.022930,0.02463,...,-0.003668,0.012370,0.01152,-0.014960,0.034730,0.020480,0.06800,0.003475,0.020140,0.000622
26006,0.028150,-0.008550,-0.04300,-0.015400,-0.022720,-0.100040,0.017260,0.087700,0.028000,0.03296,...,-0.040900,0.027100,-0.01115,0.023250,0.021960,-0.038540,0.01837,-0.022980,0.014640,0.071000


Salvar embeddings em um banco vetorial para fazer consultas

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("dados/milvus_demo.db")

In [5]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=1024,  # The vectors we will use in this demo has 768 dimensions
)

In [ ]:
data_dict = [{'id':i, 'vector': row} for i, row in df_emb.iterrows()]
data_dict

In [ ]:
res = client.insert(collection_name="demo_collection", data=data_dict)
res

In [5]:
query_vectors = model.encode(['metodologias ágeis'], batch_size=12, max_length=1024, return_dense=True, return_sparse=False, return_colbert_vecs=False)['dense_vecs']

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=10,  # number of returned entities
    output_fields=["id"],  # specifies fields to be returned
)

ids = [x['id'] for x in res[0]]
df.loc[ids]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,title,title_alt,authors,advisor1,keywords,date,publisher,campus,citation,resumo,abstract,uri,pdf_uri
unique_id,,,,,,,,,,,,,
24351,A metodologia de gestão de projetos ágeis como...,Agile project management methodology as a busi...,"[Cabau, Augusto Ferreira, Tondato, Rogério, To...","Tondato, Rogério","[Administração de projetos, Scrum (Desenvolvim...",2019-06-05,Universidade Tecnológica Federal do Paraná,Londrina,"CABAU, Augusto Ferreira; TONDATO, Rogério; TON...",As ferramentas propostas pela Metodologia de G...,The tools proposed by the Agile Project Manage...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
33896,Aplicabilidade das metodologias ágeis no geren...,Aplying agile in project management at water a...,"[Dias, Fábio Wilson]","Michaloski, Ariel Orlei","[Scrum (Desenvolvimento de software), Agile so...",2023-11-28,Universidade Tecnológica Federal do Paraná,Curitiba,"DIAS, Fabio Wilson. Aplicabilidade das metodol...",É fato que os métodos ágeis agregam às empresa...,It is a fact that agile methods add to compani...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
16837,Estudo sobre metodologias ágeis de desenvolvim...,None,"[Presner, Diego Henrique, Santos Junior, Elson...","Watanabe, Willian Massami","[Software de aplicação, Desenvolvimento ágil d...",2014-06-04,Universidade Tecnológica Federal do Paraná,Ponta Grossa,"PRESNER, Diego Henrique; SANTOS JUNIOR, Elson ...",As metodologias ágeis de desenvolvimento propo...,Agile development methodologies provide quick ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36463,Desenvolvimento de software orientado à valor ...,None,"[Rocha, Felipe Silva]","Fabri, José Augusto","[Software - Desenvolvimento, Desenvolvimento á...",2021-05-11,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"ROCHA, Felipe Silva. Desenvolvimento de softwa...",As metodologias ágeis têm sido apontadas como ...,Agile methodologies have been pointed out as a...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
25572,Diferenciais competitivos entre metodologias á...,Competitive differentials between agile and tr...,"[Ferreira, Caio Inácio Vaz]","Ferreira, Marco Antonio","[Administração de projetos, Desenvolvimento ág...",2021-05-03,Universidade Tecnológica Federal do Paraná,Londrina,"FERREIRA, Caio Inácio Vaz. Diferenciais compet...",A gestão de projetos (GP) tem recebido proposi...,Project management (PM) has received proposals...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
31370,Seleção de publicações para identificação de h...,Selection of publications to identify socio em...,"[Rodrigues, Bruno Rafael]","Figueiredo, Márcia Camilo","[Aprendizagem ativa, Aprendizagem em equipe na...",2022-12-12,Universidade Tecnológica Federal do Paraná,Londrina,"RODRIGUES, Bruno Rafael. Seleção de publicaçõe...",Este trabalho é um estudo bibliográfico. Foi e...,This assignment is a bibliographical study. A ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
34000,Aplicação da metodologia ágil Scrum em uma emp...,Agile Scrum framework application in an electr...,"[Caetano, Karina da Silva]","Carvalho, Henrique Cunha","[Scrum (Desenvolvimento de software), Empresas...",2024-06-06,Universidade Tecnológica Federal do Paraná,Campo Mourao,"CAETANO, Karina da Silva. Aplicação da metodol...",As empresas vêm buscando aprimorar o gerenciam...,Enterprises have been seeking to improve the m...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
33706,Aplicação de metodologia ágil de projeto aplic...,Application of agile project methodology appli...,"[Lopes, José Pedro]","Oliveira, Gilson Adamczuk","[Desenvolvimento ágil de software, Software li...",2023-12-01,Univ

In [8]:
from sklearn.cluster import HDBSCAN
hdb = HDBSCAN(min_cluster_size=20)
hdb.fit(df_emb)
np.unique(hdb.labels_).tolist()

[-1, 0, 1]

In [12]:
df_emb['cluster'] = hdb.labels_
df_emb['cluster']

unique_id
28320   -1
36735    0
36437   -1
36652   -1
37105    0
        ..
26646   -1
25542   -1
26006   -1
28828    0
25530    0
Name: cluster, Length: 17697, dtype: int64

In [13]:
df_emb['cluster'].value_counts()

cluster
-1    13358
 0     4317
 1       22
Name: count, dtype: int64

In [16]:
df.loc[df_emb[df_emb['cluster'] == 1].index]

,title,title_alt,authors,advisor1,keywords,date,publisher,campus,citation,resumo,abstract,uri,pdf_uri
unique_id,,,,,,,,,,,,,
29446,Estudo dos efeitos de tratamentos térmicos em ...,Study of thermal treatments on high strength l...,"[Colferai, Stella Maria Giongo]","Mazur, Viviane Teleginski","[Aço - Análise, Soldagem, Engenharia mecânica,...",2022-06-17,Universidade Tecnológica Federal do Paraná,Guarapuava,"COLFERAI, Stella Maria Giongo. Estudo dos efei...",O DOMEX 700 é um aço de baixa liga e de alta r...,DOMEX 700 is a low alloy steel with high mecha...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
29044,Avaliação do efeito de 3 diferentes refinadore...,Evaluation of the effect of 3 different commer...,"[Pasquini, Lucas, Giacoia, Tamires Russo]","Lourençato, Luciano Augusto","[Metalografia, Ligas de alumínio, Metalurgia, ...",2021-12-10,Universidade Tecnológica Federal do Paraná,Ponta Grossa,"PASQUINI, Lucas; GIACOIA, Tamires Russo. Avali...",O alumínio é um dos elementos mais abundantes ...,Aluminum is one of the most abundant elements ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
35293,Resistência ao riscamento do aço AISI 420 nitr...,Scratch resistance of AISI 420 nitrided at low...,"[Souza, Andrey Serafin de]","Bernardelli, Euclides Alexandre","[Aço inoxidável - Tratamento térmico, Nitruraç...",2022-11-29,Universidade Tecnológica Federal do Paraná,Curitiba,"SOUZA, Andrey Serafin de. Resistência ao risca...","No ramo da engenharia, o estudo para aprimorar...","At engineering field, the study to improve cer...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
35282,Estudo das propriedades da liga de alumínio 70...,Study of the Properties of the heat treated 70...,"[Santos, Yuri Alex dos]","Slva, Carla Dantas da","[Alumínio, Termodinâmica, Engenharia mecânica,...",2024-08-14,Universidade Tecnológica Federal do Paraná,Guarapuava,"SANTOS, Yuri Alex dos. Estudo das propriedades...",O alumínio é o segundo metal mais produzido at...,Aluminum is the second most produced metal tod...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
24515,Análise de falha em componente de motor veicular,Failure analysis in engine component,"[Gomes, Isabel Cristina]","Verástegui, Roger Navarro","[Automóveis - Motores, Ligas de alumínio, Comp...",2019-12-05,Universidade Tecnológica Federal do Paraná,Ponta Grossa,"GOMES, Isabel Cristina. Análise de falha em co...",Atualmente eles vêm sendo largamente utilizado...,"Currently they have been widely used, mainly i...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
5188,Planejamento experimental aplicando Taguchi em...,Experimental planning applying Taguchi in a 30...,"[Silva, Marcos Vinícius Tavares da]","Moreno, Joao Roberto Sartori","[Soldagem, Revestimento em metal, Aço inoxidáv...",2019-07-15,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"SILVA, Marcos Vinícius Tavares da. Planejament...",A soldagem de revestimento de aços carbono com...,Welding coating of carbon steels with stainles...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
27559,Influência dos parâmetros de soldagem de reves...,None,"[Cardim, Pedro Peres]","Corrêa, Celso Alves","[Soldagem, Aço, Revestimentos, Welding, Steel,...",2018-12-07,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"CARDIM, Pedro Peres. Influência dos parâmetros...",A soldagem de revestimento com aço inox vem se...,Stainless steel cladding has been widely used ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
10426,Nitretação por plasma do aço inoxidável ISO 58...,Plasma nitriding of ISO 5832-1 stainless steel...,"[Rubega Neto, Alfredo, Marques, Camila Gonçalves]","Bernardell